# Ejemplo final

Implementamos el mismo ejemplo anterior pero esta vez utilizando **cluster sharding**.

## Índice
- Protocolo de mensajería
  + De entrada
  + De salida
  + Eventos
- Cuentas y transferencias
  + Actor cuenta
    - Funciones, a partir del mensaje: 
      + Id de cuenta a partir del mensaje
      + Región de sharding a partir del mensaje
  + Interfaz cuenta
  + Definición de una transferencia
- Implementaciones
  + Logica de negocio: actualización del balance
  + Publicación de eventos
- Probandolo todo
  + Se crea el sistema de actores con la configuración necesaria
  + Se crea la región de cluster sharding
- Bonus track
  + Utilizar `akka-management`
  + Escuchar los eventos del cluster 

###  Se importan la librerías de akka 

> Ahora se importan las librerías de cluster sharding

In [ ]:
import $ivy.`com.typesafe.akka::akka-cluster-sharding:2.5.14`
import $ivy.`com.lightbend.akka.management::akka-management:0.17.0`
import $ivy.`com.lightbend.akka.management::akka-management-cluster-http:0.17.0`

### Implicitos necesarios

In [ ]:
import scala.concurrent.ExecutionContext 
import java.util.concurrent.Executors
import akka.util.Timeout
import scala.concurrent.duration._

implicit val ec  = ExecutionContext.fromExecutorService( Executors.newFixedThreadPool( 10 ) )
implicit val timeout = Timeout( 5 seconds )   

object Types {
    
   type Balance = Int
   type Amount  = Int
   type IdAccount = String
}


---

---
## Se define el protocolo de mensajería

### Protocolo de entrada

In [ ]:
import Types._

sealed trait AccountIn {
    val accountId : IdAccount
}

// Commands

sealed trait AccountCommand extends AccountIn { 
    val amount : Amount
}

final case class Withdrawal(accountId : IdAccount, amount : Amount) extends AccountCommand 
final case class Income(accountId : IdAccount, amount : Amount) extends AccountCommand 


// Queries

sealed trait AccountQuery extends AccountIn

final case class GetBalance(accountId : IdAccount) extends AccountQuery

### Protocolo de salida

In [ ]:
import Types._ 

sealed trait AccountOut

final case class CurrentBalance( balance: Balance ) extends AccountOut

### Eventos

In [ ]:
import Types._ 

sealed trait AccountEvent {    
    val idAccount : IdAccount
    val amount: Amount
}

case class WithdrawalCreated( 
                                val idAccount: IdAccount, 
                                val amount : Amount
                            ) extends AccountEvent
case class IncomeCreated( 
                            val idAccount: IdAccount, 
                            val amount : Amount
                        ) extends AccountEvent

---

---
## Cuentas y transferencias
### Actor 'Cuenta'

In [ ]:
import akka.actor._
import scala.collection.mutable.Queue
import scala.util._
import Types._

class ActorAccount( 
                    private val updateBalance : (Amount, Balance) => Try[Balance], 
                    private val queueCQRS: Queue[AccountEvent] 
                 ) extends Actor {
    
    val id = self.path.name
    
    var balance : Int = 0
    
    override def receive = {
        
        case command : AccountCommand => manageCommads( command )
        case querry  : AccountQuery   => manageQueries( querry )
        case other                    => unhandled( other )
        
    }
    
    private def manageCommads( command: AccountCommand ) : Unit = {
        
        command match {
            case Withdrawal(_, amount ) => execUpdateBalance( -1 * amount, command)
            case Income( _, amount )     => execUpdateBalance( amount, command )
            
        }           
        
    }
    
    private def execUpdateBalance( amount : Amount, command : AccountCommand) = {        
        updateBalance( amount, balance ) match {            
            case Success( newBalance ) => {
                balance = newBalance
                responseBalance
                sendEvent( command )
            }
            case Failure(  error ) => sender() ! Status.Failure( error )         
        }
    }
    
    private def sendEvent( command: AccountCommand ) {
        
        val event : AccountEvent = command match {
            case Withdrawal(_, amount ) => WithdrawalCreated( id, amount ) 
            case Income( _, amount )     => IncomeCreated( id, amount ) 
        }
        
        queueCQRS.enqueue( event )
        
    }
    
    private def manageQueries( queries : AccountQuery ) : Unit = queries match {
        case GetBalance(_) => sender() !  CurrentBalance( balance )
    }
    
    private def responseBalance() = sender() !  CurrentBalance( balance )    
}


/*

#### Se crea un objeto acompañante
Tendra las funciones necesarias para la creación de la región de sharding
*/

object ActorAccount {
    
    import akka.cluster.sharding.{ShardRegion, ClusterSharding, ClusterShardingSettings}
    import akka.actor.ActorRef
    
    
    def props( updateBalance : (Amount, Balance) => Try[Balance], queueCQRS: Queue[AccountEvent] ) = Props {
        new ActorAccount( updateBalance, queueCQRS ) 
    }        
    
    def extractShardId: ShardRegion.ExtractShardId = {
        case a : AccountIn =>  ( Math.abs( a.accountId.hashCode % 3 ) ).toString

    }

    def  extractEntityId: ShardRegion.ExtractEntityId = {
        case a : AccountIn =>  ( a.accountId, a)
    }
    
}


### Interfaz 'Cuenta'

In [ ]:
import scala.concurrent.Future
import Types._

trait Account[P[_]] {
   def makeWithdrawal( amount : Amount ) : P[Balance] 
   def makeIncome( amount : Amount ) : P[Balance] 
   def getBalance: P[Balance]
}

object Account {
    
    import akka.pattern._
    import akka.actor._
    import akka.util.Timeout
    
    private def toBalance( responseActor : Future[Any] )( implicit ec : ExecutionContext ) : Future[Balance] = {
        responseActor.mapTo[CurrentBalance].map( _.balance )
    }
    
    def apply( accountId : String, accountSharding : ActorRef )
                ( implicit ec : ExecutionContext, timeout : Timeout ) = new Account[Future] {
      
         def makeWithdrawal( amount : Int ) : Future[ Int] = toBalance {
            accountSharding ? Withdrawal( accountId, amount )
         }

         def makeIncome( amount : Int ) : Future[Int] = toBalance {
             accountSharding ? Income( accountId,  amount ) 
         }

         def getBalance : Future[Int] = toBalance {
             accountSharding ? GetBalance( accountId )
         }
        
    }
}



### Defincición de una transferencia

Se simula una operación/compensacion siguiendo el patrón sagas

In [ ]:
import Types._

object Transfer {
    
    import scala.concurrent._
    
    def transfer( from : Account[Future], to: Account[Future] )( amount : Amount )( implicit ec : ExecutionContext) = {
        
        for {
            
             _  <- from.makeWithdrawal( amount ) 
            res <- to.makeIncome( amount )
                    .map( _ => true )
                    .recoverWith{ 
                        case _ => from.makeIncome( amount ).map( _=> false ) 
                    }
        } yield( res )
        
   }
    
}

---

----
## Implementaciones

### Lógica de negocio
Se define una lógica de negocio simple. En este caso no se admiten descubiertos, pero por ejemplo se pueden implementar diferentes lógicas como un porcentaje de descubierto dependiendo del balance. 
> El objetivo final es que la lógica puede estar separada del actor y puede ser validada y probada aparte

In [ ]:
import scala.util._
import Types._

implicit val updateBalance : (Amount,Balance) => Try[Balance] = ( amount, balance ) => {
   
    val newBalance = amount + balance
    
    if( newBalance >= 0 ) {
    
        Success( newBalance )
        
    } else {
        
        Failure( new IllegalStateException( s"It should not be in red( ${newBalance} )" ) )
    }
    
}

### Indirección de publicación de eventos
Se define una cola que será la indirección de publicación de eventos.
En este caso para esta prueba será una cola mitable de Scala.   
> En un sistema real puede ser un akka stream con su fuente '_materializada_' en una cola

In [ ]:
import scala.collection.mutable.Queue

val queueCQRS = Queue[AccountEvent]()

---

---
## Probandolo todo

### _Testing: Utilidades_

> **Sólo para motivos de testing**. Espera el resultado de un futuro

In [ ]:
object TestUtil {
    
    import scala.concurrent._, duration._
    import akka.pattern._
    import akka.util.Timeout


    val tm = 5 seconds
    val timeout = Timeout( tm )

    def result[T]( future : => Future[T] ) =  Try {
        Await.result( future, tm )
    }
    
}

---
### Iniciando el entorno

#### Singlenton de utilidades del sistema de actores

Se crea un objeto con los métodos necesarios para crear el sistema de actores con la configuración necesaria que requiere akka sharding. También permite parar el sistema de actores de una manera ordenada.

> En este caso existen dos _seed nodes_ configurados para permitir comprobar el comportamiento del cluster

In [ ]:

object SystemUtil {
    
    import com.typesafe.config.ConfigFactory 
    import akka.actor._
    import akka.cluster.Cluster
    import scala.concurrent.Future

    val SystemName = "test2"
    
    val AkkaPort = 2554
    
    val AkkaManagementPort = 8554
    
    val akkaCfg =
      s"""
        |akka {
        |  
        |  remote {
        |    netty.tcp {
        |      hostname = "127.0.0.1"
        |      port = ${AkkaPort}
        |    }
        |  }
        |  
        |  cluster {
        |     seed-nodes = [
        |                     "akka.tcp://${SystemName}@127.0.0.1:2554",
        |                     "akka.tcp://${SystemName}@127.0.0.1:2553"
        |                   ]
        |      sharding.state-store-mode = ddata
        |    }
        |
        |  actor {
        |    provider = "akka.cluster.ClusterActorRefProvider"
        |  }
        |
        |  management {
        |     http {
        |       hostname = "127.0.0.1"
        |       port = ${AkkaManagementPort} 
        |     }
        |
        |  }
        |
        |}
      """.stripMargin
    
    lazy val system = ActorSystem.create( SystemName, 
                                          ConfigFactory.parseString( akkaCfg ).resolve() 
                                         )
    def terminate : Unit = {
        val cluster = Cluster.get( system )
        cluster.registerOnMemberRemoved( system.terminate )
        cluster.leave( cluster.selfAddress )
    }
    
}



####  Se crea la región de cluster sharding

In [ ]:
import akka.cluster.sharding.{ClusterSharding, ClusterShardingSettings}
import akka.actor.ActorRef

val accountsSharding : ActorRef = ClusterSharding( SystemUtil.system ).start(
      typeName = "accounts",
      entityProps = ActorAccount.props( updateBalance, queueCQRS ),
      settings = ClusterShardingSettings( SystemUtil.system ),
      extractShardId = ActorAccount.extractShardId,
      extractEntityId = ActorAccount.extractEntityId
    )

#### Se crean dos 'entidades' cuenta

In [ ]:
val accountOne = Account( "accountOne", accountsSharding )
val accountTwo = Account( "accountTwo", accountsSharding )

#### Se hace un ingreso incial a las dos cuentas

Se obtiene el resutlado de los dos balances y se calcula el total del dinero (la suma de los dos balances)

> Aquí se hace `Await` sólo por motivos de testing

In [ ]:

TestUtil.result{
    
    accountOne.makeIncome( 1000 ).zipWith( accountTwo.makeIncome( 1000 ) ){
        ( b1, b2 ) => (b1, b2, b1 +b2) 
    }
    
}

#### Se comprueban los eventos

In [ ]:
queueCQRS.toList ; queueCQRS.clear

---
### _Probando, probando_

#### Funciones de utilidades
Para poder testear transferencias de una cuenta a otra de una manera más cómoda

In [ ]:
val transfersOneToTwo =  Transfer.transfer( accountOne, accountTwo)( _ )
val transfersTwoToOne =  Transfer.transfer( accountTwo, accountOne)( _ )

#### Primera prueba

Dos transferencias lanzadas en paralelo. El mismo importe (`500`) desde la cuenta 1 a la 2 y desde la 2 a la 1

In [ ]:
TestUtil.result{
    
    transfersOneToTwo( 500 ).zipWith( transfersTwoToOne( 500 ) ) {
        (a, b) =>  a && b
    }
    
}

#### Se vuelen a compruebar los balances

> `Await` sólo por motivos de testing

In [ ]:
TestUtil.result {
    accountOne.getBalance.zipWith( accountTwo.getBalance ){
        ( b1, b2 ) => (b1, b2, b1 +b2) 
    }
}

#### Se vuelven a comprobar los eventos

In [ ]:
queueCQRS.toList ; queueCQRS.clear

---
### Bonus track


#### Akka management

Se arranca akka-management. En este caso arranca un api rest en el puerto definido en `AkkaManagementPort` en `SystemUtil`

In [ ]:
import akka.management.AkkaManagement

TestUtil.result {
    AkkaManagement( SystemUtil.system ).start()    
}

#### Escuchar eventos del estado del cluster

Actor '_listener_' que escucha los eventos del cluster `MemberEvent` y `ReachabilityEvent` y los almacena en una variable.

Se puede obtener esa información eviando un mensaje del tipo `GetClusterStateEvent`. Después de devolver estos datos se incializa la variable.

In [ ]:
sealed trait EventClusterListenerIn 
final case object GetClusterStateEvent extends EventClusterListenerIn

class EventClusterListener extends Actor {
    
    import akka.cluster.Cluster
    
    import akka.cluster.ClusterEvent._
    
    val cluster = Cluster( context.system  )
    
    cluster.subscribe(self, 
                      initialStateMode = InitialStateAsEvents, 
                      classOf[MemberEvent], 
                      classOf[ReachabilityEvent] )
    
    var listDomain = Set[ClusterDomainEvent]()       
    
    
    override def receive = {
        
        case a : ClusterDomainEvent => {
            listDomain = listDomain + a
        }
        
        case GetClusterStateEvent => {
            sender() ! listDomain
            listDomain = Set()
        }       
    }
    
}


Se crea un objeto que envuelve al actor para gestionar estos mensajes

In [ ]:
object EventClusterListener {
    
    import akka.pattern._
    import akka.actor._
    import akka.util.Timeout
    
    import akka.cluster.ClusterEvent._
    
    lazy val listener = SystemUtil.system.actorOf( Props( new EventClusterListener() ) )
    
     def getEvents: Future[Set[ClusterDomainEvent]] = {
       ( listener ? GetClusterStateEvent ) .mapTo[Set[ClusterDomainEvent]]           
     }
}



Se obtienes los eventos escuchados por este nodo

> Se utiliza `TestUtil` por motivos de testing

In [ ]:
TestUtil.result {
   EventClusterListener.getEvents
}